In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF,  LatentDirichletAllocation
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

from nltk import sent_tokenize
from nltk.corpus import stopwords

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import pandas as pd
import numpy as np

import string
import spacy
import gzip
import simplejson as json
import nltk
import en_core_web_sm
nlp = en_core_web_sm.load()

from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import csv

d:\pycharmprojects\venv\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
#Script for SAVING DATA. Uncomment it when needed

"""import os
import gzip
import json

def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield json.loads(l)

fileName = 'Electronics_5'
inputFileName  = fileName + '.json.gz'
if not os.path.exists(fileName): os.makedirs(fileName)

count = 0
ii = 0
data = {}
asin = '0'
for review in parse(inputFileName):
    if review['asin'] != asin:
        outputFileName = fileName + r'/' + asin + '.json'
        outputFile = open(outputFileName, 'w', newline='')
        json.dump(data, outputFile)
        outputFile.close
        ii = 0
        data = {}
        asin = review['asin']
    try: helpfulness = review['vote']
    except: helpfulness = '0'
    try:
        data[ii] = {
            'helpfulness': helpfulness,
            'rating': review['overall'],
            'text': review['reviewText']}
        ii += 1
    except: pass
    count += 1
    if count % 1e5 == 0: print(count)"""

"import os\nimport gzip\nimport json\n\ndef parse(path):\n    g = gzip.open(path, 'r')\n    for l in g:\n        yield json.loads(l)\n\nfileName = 'Electronics_5'\ninputFileName  = fileName + '.json.gz'\nif not os.path.exists(fileName): os.makedirs(fileName)\n\ncount = 0\nii = 0\ndata = {}\nasin = '0'\nfor review in parse(inputFileName):\n    if review['asin'] != asin:\n        outputFileName = fileName + r'/' + asin + '.json'\n        outputFile = open(outputFileName, 'w', newline='')\n        json.dump(data, outputFile)\n        outputFile.close\n        ii = 0\n        data = {}\n        asin = review['asin']\n    try: helpfulness = review['vote']\n    except: helpfulness = '0'\n    try:\n        data[ii] = {\n            'helpfulness': helpfulness,\n            'rating': review['overall'],\n            'text': review['reviewText']}\n        ii += 1\n    except: pass\n    count += 1\n    if count % 1e5 == 0: print(count)"

In [3]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')


class CleanTextTransformer(TransformerMixin):
   
    def transform(self, X, **transform_params):
        #return [cleanText(text) for text in X]
        return [text for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}
    
"""def cleanText(text):
    "this function removes new lines."
    text = text.strip().replace("\n", " ").replace("\r", " ")
    return text
"""

def tokenizeText(sample):
    "This function tokenizes text and does other preprocessing steps like Lemmatization and Stemming."

    tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
    stop_words = set(stopwords.words('english')) 
    lemmatizer = WordNetLemmatizer()
    
    #tokenize
    tokens = tokenizer.tokenize(sample)
    # lemmatize
    lemmas = []
    for word in tokens:
        if word.isalnum() and not word in stop_words:
            word = word.lower()
            word = lemmatizer.lemmatize(word, pos = 'v')
            lemmas.append(word)
    tokens = lemmas
    # white space removal and new line removal
    while "" in tokens:
        tokens.remove("")
    while " " in tokens:
        tokens.remove(" ")
    while "\n" in tokens:
        tokens.remove("\n")
    while "\n\n" in tokens:
        tokens.remove("\n\n")

    return tokens

#------------------------------------------------------------------------------
def return_topics(vectorizer, clf, W, df, n_top_words, n_top_documents):
    print('return topics')
    topics, reviews = [], []
    features = vectorizer.get_feature_names()
    sentiment_analyser = SentimentIntensityAnalyzer()

    for topic_id, topic in enumerate(clf.components_):

        # grab the list of words describing the topic
        topic_word_list = []
        for i in topic.argsort()[:-n_top_words - 1:-1]:
            topic_word_list.append(features[i])

        # split words in case there are some bigrams
        split_topic_word_list = []
        for word in topic_word_list:
            for splitted in word.split():
                split_topic_word_list.append(splitted)
        topic_words = list(set(split_topic_word_list))

        # append topic words as a single string
        topics.append(' '.join([word for word in topic_words]))

        # iterate for reviews for each topic
        topic_doc_indices = np.argsort(W[:, topic_id])[::-1][0:n_top_documents]

        for doc_ind in topic_doc_indices:
            review = df['reviewText'].iloc[doc_ind]

            # check if the review contains any of the topic words
            if any(word in review.lower() for word in topic_words):
                # analyse sentiment
                vader = sentiment_analyser.polarity_scores(review)
                # form the review - topic_id and sentiment data structure
                reviews.append(df.iloc[doc_ind].to_dict())
                reviews[-1]['topic'] = topic_id
                reviews[-1]['sentiment'] = vader['compound']

    return topics, reviews
#------------------------------------------------------------------------------

def summarize_reviews(topics, reviews):
    # topics: list of strings. Each string contains the topics for a review
    # reviews: list of dicts with the following fields
    #  'reviewText': string with text of the review
    #  'topic': topics index
    # returns reviews with the following new fields
    #  'summary': sentences from review w/ topic words

    analyser = SentimentIntensityAnalyzer()
    summary_all_review = []
    for ii, review in enumerate(reviews):
        summary = []
        sentences = sent_tokenize(review['reviewText'])
        topic_words = topics[review['topic']].split()

        for sentence in sentences:
            for word in topic_words:
                if word in sentence.lower():
                    summary.append(sentence)
                    break

        reviews[ii]['summary'] = ' '.join([sentence for sentence in summary])
        vader = analyser.polarity_scores(reviews[ii]['summary'])
        reviews[ii]['summary_sentiment'] = vader['compound']
        
        summary_all_review.append(reviews[ii]['summary'])

    return reviews, summary_all_review

def print_topics(test_asin):

    test_df = reviews_df[reviews_df['asin'] == test_asin].dropna()
    n_features, n_top_words, n_topics, n_top_documents = 1000, 3, 8, 3

    vectorizer = TfidfVectorizer(max_features=n_features,
                                 tokenizer=tokenizeText,
                                 stop_words='english',
                                 ngram_range=(1,2))

    clf = NMF(n_components=n_topics, random_state=1, solver='mu', beta_loss='frobenius')
   
    #clf = LatentDirichletAllocation(n_components = 5, max_iter = 5, learning_method ='online',learning_offset = 50.,random_state = 0)

    pipe = Pipeline([('cleanText', CleanTextTransformer()),('vectorizer', vectorizer), ('nmf', clf)])

    pipe.fit(test_df['reviewText'])
    transform = pipe.fit_transform(test_df['reviewText'])
    
    #topic identification
    topics, reviews = return_topics(vectorizer, clf, transform, test_df, n_top_words, n_top_documents)
    # review summarization
    reviews , summary = summarize_reviews(topics, reviews)
    print("Summary :\n", len(summary))
    print("Topics:", len(topics))
    
    return topics, reviews

In [29]:
#reviews_df = getDF('Video_Games_5.json.gz')
reviews_df = getDF('Electronics_5.json.gz')
print(reviews_df.head(4))

       reviewerID        asin              reviewerName   helpful  \
0   AO94DHGC771SJ  0528881469                   amazdnu    [0, 0]   
1   AMO214LNFCEI4  0528881469           Amazon Customer  [12, 15]   
2  A3N7T0DY83Y4IG  0528881469             C. A. Freeman  [43, 45]   
3  A1H8PY3QHMQQA0  0528881469  Dave M. Shaw "mack dave"   [9, 10]   

                                          reviewText  overall  \
0  We got this GPS for my husband who is an (OTR)...      5.0   
1  I'm a professional OTR truck driver, and I bou...      1.0   
2  Well, what can I say.  I've had this unit in m...      3.0   
3  Not going to write a long review, even thought...      2.0   

                   summary  unixReviewTime   reviewTime  
0          Gotta have GPS!      1370131200   06 2, 2013  
1        Very Disappointed      1290643200  11 25, 2010  
2           1st impression      1283990400   09 9, 2010  
3  Great grafics, POOR GPS      1290556800  11 24, 2010  


In [5]:
def max_entropy(n):
    return -np.log(1/n)

def unique(sequence):
    '''get unique elements of list and keep the same order'''
    
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

def redundancy(string):
    entropy, string_list = 0, string.split()
    string_set = unique(string_list)
    for word in string_set:
        p = string_list.count(word)/len(string_list)
        entropy -= p*np.log(p)        
    return 1 - entropy/max_entropy(len(string_list))

def lemmatize(string):
    from nltk.stem import PorterStemmer
    stemmer = PorterStemmer()
    tokens = parser(string)
    lemmas = []
    for token in tokens:
        lemmas.append(stemmer.stem(token.lemma_.lower().strip()))
        
    return ' '.join(lemma for lemma in lemmas)


In [6]:
string = 'speaker system sound quality headphone jack satellite speakers sound card highly recommend thx certified midrange bang for the buck living room home theater altec lansing'
lemmas = tokenizeText(string) 
lemmas = ' '.join(lemma for lemma in lemmas)
print(lemmas)
print(redundancy(lemmas))

topics, reviews = print_topics('B0002SQ2P2')

print(topics)
# print(emoji_topics)
lemmas = ' '.join(lemma for lemma in topics)
print(lemmas)
print(redundancy(lemmas))

speaker system sound quality headphone jack satellite speakers sound card highly recommend thx certify midrange bang buck live room home theater altec lansing
0.019223019952826492


d:\pycharmprojects\venv\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['make'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


return topics
Summary :
 24
Topics: 8
['bass sound music', '2300 z', 'great sound', 'buy speakers computer', 'remote satellite cable', 'sound good card', 'work great bose', 'speaker best set']
bass sound music 2300 z great sound buy speakers computer remote satellite cable sound good card work great bose speaker best set
0.06885190025657495


In [7]:
string = 'earbuds sound quality couple months every 6 months sound was good great sound bass volume ears earbud earphones hear inexpensive jack model broke cord'
lemmas = tokenizeText(string) 
lemmas = ' '.join(lemma for lemma in lemmas)
print(lemmas)
print(redundancy(lemmas))



topics, review = print_topics('B0002D03ZW')
print(topics)
lemmas = ' '.join(lemma for lemma in topics)
print(lemmas)
print(redundancy(lemmas))

earbuds sound quality couple months every 6 months sound good great sound bass volume ears earbud earphones hear inexpensive jack model break cord
0.06492466861659307


d:\pycharmprojects\venv\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['make'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


return topics
Summary :
 24
Topics: 8
['headphones sound music', 'years home studio', 'head wear ears', 'comfortable great', 'flat hz response', 'price expect feel', 'read review say', 'amp headphone fit']
headphones sound music years home studio head wear ears comfortable great flat hz response price expect feel read review say amp headphone fit
0.0


In [8]:
from sklearn.model_selection import GridSearchCV

def grid_search_topic_model(test_asin):
    search_params = {'n_components': [3,4,5,6,7,8,9,10,11,12], "solver": ['cd','mu']}
    test_df = reviews_df[reviews_df['asin'] == test_asin].dropna()
    n_features, n_top_words, n_topics, n_top_documents = 1000, 3, 6, 3
    vectorizer = TfidfVectorizer(max_features=n_features,
                                 tokenizer=tokenizeText,
                                 stop_words='english',
                                 ngram_range=(1,2))

    clf = NMF()

    pipe = Pipeline([('cleanText', CleanTextTransformer()),('vectorizer', vectorizer)])

    # pipe.fit(test_df['reviewText'])
    data_vectorized = pipe.fit_transform(test_df['reviewText'])

    model = GridSearchCV(clf, param_grid=search_params)

    model.fit(data_vectorized)
    best_topic_model = model.best_estimator_
    print("Best Model's Params: ", model.best_params_)

In [54]:
grid_search_topic_model('B0002SQ2P2')

d:\pycharmprojects\venv\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['make'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=None, random_state=None, shuffle=False, solver='cd',
    tol=0.0001, verbose=0) does not.

LDA Best Model's Params:  {'learning_decay': 0.5, 'n_components': 3}

In [9]:
def print_topics_lda(test_asin):

    test_df = reviews_df[reviews_df['asin'] == test_asin].dropna()
    n_features, n_top_words, n_topics, n_top_documents = 1000, 3, 8, 3

    vectorizer = TfidfVectorizer(max_features=n_features,
                                 tokenizer=tokenizeText,
                                 stop_words='english',
                                 ngram_range=(1,2))

    clf = NMF(n_components=n_topics, random_state=1, solver='mu', beta_loss='frobenius')
   
    clf = LatentDirichletAllocation(n_components = 3, max_iter = 5, learning_method ='online',learning_offset = 50.,random_state = 0)

    pipe = Pipeline([('cleanText', CleanTextTransformer()),('vectorizer', vectorizer), ('nmf', clf)])

    pipe.fit(test_df['reviewText'])
    transform = pipe.fit_transform(test_df['reviewText'])
    
    #topic identification
    topics, reviews = return_topics(vectorizer, clf, transform, test_df, n_top_words, n_top_documents)
    # review summarization
    summary = summarize_reviews(topics, reviews)
    #print("Summary :\n", summary)
    print("Topics:")
    
    return topics, reviews

In [10]:
string = 'speaker system sound quality headphone jack satellite speakers sound card highly recommend thx certified midrange bang for the buck living room home theater altec lansing'
lemmas = tokenizeText(string) 
lemmas = ' '.join(lemma for lemma in lemmas)
print(lemmas)
print(redundancy(lemmas))



topics, review = print_topics_lda('B0002SQ2P2')
print(topics)
lemmas = ' '.join(lemma for lemma in topics)
print(lemmas)
print(redundancy(lemmas))

speaker system sound quality headphone jack satellite speakers sound card highly recommend thx certify midrange bang buck live room home theater altec lansing
0.019223019952826492


d:\pycharmprojects\venv\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['make'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


return topics
Topics:
['speakers better like', 'speakers sound great', 'bass speakers sound']
speakers better like speakers sound great bass speakers sound
0.23676997261905086


In [11]:
test_amazon_asins = ['B0002SQ2P2', 'B0002KVQBA', 'B00029MTMQ','B00020S7XK','B00063E2HS','B0002D03ZW','B0002WPSBC','B00006JN3G','B0002CZHN6','B00004T8R2','B00004ZCJJ',
'B00007M1TZ','B0002UPGOI','B000204SWE','B0002EMY9Y','B00006IAKJ','B000629GES','B00017LSPI','B0002UM0JW',
'B0000C3GWU','B0001NNLHK','B0000BYDKO','B00008MOPJ','B00066HP7Y','B0000AQIFW','B00066EK2W','B00005NIMJ',
'B00009WQS1','B0000DJEK7','B00028D778','B00030CHRQ','B0002IOIMQ','B0001EMA80','B00006JILE','B0002Y5WXO',
'B00062UW5A','B00007GQLU','B00004TS16','B00005QXWI','B00018MSNI']

test_elecronics5_asins = reviews_df['asin']
print(list(test_elecronics5_asins[:5]))

def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2))

print(intersection(test_amazon_asins, test_elecronics5_asins)) 

valid_asins = intersection(test_amazon_asins, test_elecronics5_asins)

print(valid_asins)

['0528881469', '0528881469', '0528881469', '0528881469', '0528881469']
['B00030CHRQ', 'B0002Y5WXO', 'B0002IOIMQ', 'B00007M1TZ', 'B00062UW5A', 'B000629GES', 'B0000DJEK7', 'B00006JN3G', 'B0000BYDKO', 'B0001NNLHK', 'B00020S7XK', 'B0002UM0JW', 'B0002UPGOI', 'B0002SQ2P2', 'B00005QXWI', 'B00017LSPI', 'B0000C3GWU', 'B0002D03ZW', 'B0002WPSBC', 'B00028D778', 'B00066EK2W', 'B00004T8R2', 'B0002CZHN6', 'B00004TS16', 'B000204SWE', 'B00009WQS1', 'B00066HP7Y', 'B0000AQIFW', 'B00005NIMJ', 'B00007GQLU', 'B00018MSNI', 'B0002KVQBA']
['B00030CHRQ', 'B0002Y5WXO', 'B0002IOIMQ', 'B00007M1TZ', 'B00062UW5A', 'B000629GES', 'B0000DJEK7', 'B00006JN3G', 'B0000BYDKO', 'B0001NNLHK', 'B00020S7XK', 'B0002UM0JW', 'B0002UPGOI', 'B0002SQ2P2', 'B00005QXWI', 'B00017LSPI', 'B0000C3GWU', 'B0002D03ZW', 'B0002WPSBC', 'B00028D778', 'B00066EK2W', 'B00004T8R2', 'B0002CZHN6', 'B00004TS16', 'B000204SWE', 'B00009WQS1', 'B00066HP7Y', 'B0000AQIFW', 'B00005NIMJ', 'B00007GQLU', 'B00018MSNI', 'B0002KVQBA']


In [12]:
red_nmf = 0 

redundancy_arr = ()

for asin in valid_asins:
    topics, review = print_topics(asin)
    print(asin,topics)
    lemmas = ' '.join(lemma for lemma in topics)
    temp_red = redundancy(lemmas)
    red_nmf += temp_red
    redundancy_arr = list(redundancy_arr)
    redundancy_arr.append((asin,temp_red))
    redundancy_arr = tuple(redundancy_arr)
    
red_lda = 0 

for asin in valid_asins:
    topics, review = print_topics_lda(asin)
    print(asin, topics)
    lemmas = ' '.join(lemma for lemma in topics)
    red_lda += redundancy(lemmas)

d:\pycharmprojects\venv\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['make'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


return topics
Summary :
 24
Topics: 8
B00030CHRQ ['extension cord length', 'buy earbuds ears', 'headphones great ear', 'expose coat wire', 'bass buy sony', 'work pull ear', 'price use', 'mp3 player']


d:\pycharmprojects\venv\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['make'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


return topics
Summary :
 24
Topics: 8
B0002Y5WXO ['lens kit great', 'zoom light good', 'lense print good', 'canon tamron mm', 'repair problem canon', 'purpose lens general', 'return great amazon', 'heavy picture big']


d:\pycharmprojects\venv\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['make'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


...

return topics
Topics:
B00007GQLU ['price prime lens great', 'great lens reasonable', 'lens f great']


d:\pycharmprojects\venv\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['make'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


return topics
Topics:
B00018MSNI ['headphone headphones sound', 'close headphones replace', 'headphones sound like']


d:\pycharmprojects\venv\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['make'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


return topics
Topics:
B0002KVQBA ['sound sub subwoofer', 'room repair sub', 'huummm subwoofer']


In [13]:
print(redundancy_arr)

(('B00030CHRQ', 0.04077160440319583), ('B0002Y5WXO', 0.07270143066184376), ('B0002IOIMQ', 0.0694134639479278), ('B00007M1TZ', 1.1102230246251565e-16), ('B00062UW5A', 0.07689207981130575), ('B000629GES', 0.057669059858479255), ('B0000DJEK7', 4.440892098500626e-16), ('B00006JN3G', 0.0694134639479278), ('B0000BYDKO', 0.10192901100798912), ('B0001NNLHK', -2.220446049250313e-16), ('B00020S7XK', 0.06504864248484399), ('B0002UM0JW', 0.03844603990565287), ('B0002UPGOI', 0.016365081042566065), ('B0002SQ2P2', 0.06885190025657495), ('B00005QXWI', 0.03890705966059538), ('B00017LSPI', 0.04627564263195183), ('B0000C3GWU', 0.13009728496968764), ('B0002D03ZW', 0.0), ('B0002WPSBC', 0.07814669001728858), ('B00028D778', 0.0), ('B00066EK2W', 0.02313782131597597), ('B00004T8R2', 0.04077160440319594), ('B0002CZHN6', 0.08923770245817264), ('B00004TS16', 0.059432742193288424), ('B000204SWE', 0.020385802201598135), ('B00009WQS1', 0.020385802201598135), ('B00066HP7Y', 0.06546032417026526), ('B0000AQIFW', 4.4408

In [14]:
asins_count = len(valid_asins)

avd_red_nmf = red_nmf/asins_count
avg_red_lda = red_lda/asins_count

print('Avg reduncdancy NMF:', avd_red_nmf)
print('Avg reduncdancy LDA:', avg_red_lda)

Avg reduncdancy NMF: 0.04757272814138594
Avg reduncdancy LDA: 0.14086402562112307


In [15]:
scraped_tags_df = pd.read_csv('amazon_scraped_tags.csv',keep_default_na=False)

In [16]:
print(scraped_tags_df.columns)

tags_df = scraped_tags_df.iloc[: , 2 : 20]
tags_df.head(5)

tags_df['CombinedTags'] = tags_df[tags_df.columns[1:]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)
asin_tags_df = tags_df.loc[ : , ['ASIN','CombinedTags'] ]
asin_tags_df= asin_tags_df[asin_tags_df.ASIN.isin(valid_asins)]
asin_tags_df.to_csv('valid_asin_scraped_tags_df.csv')

asin_tags_df.head(5)

Index(['Product', '# of Reviews', 'ASIN', 'Tags', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
       'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18',
       'Unnamed: 19'],
      dtype='object')


,ASIN,CombinedTags
0,B0002SQ2P2,speaker system sound quality headphone jack sa...
1,B0002KVQBA,polk audio living room sounds great surround s...
3,B00020S7XK,great little pocket radio battery life sound q...
5,B0002D03ZW,earbuds sound quality couple months every 6 mo...
6,B0002WPSBC,home theater surround sound speaker system sou...


In [17]:
valid_asins = list(asin_tags_df['ASIN'])
print(valid_asins)
review_sentences = list(asin_tags_df['CombinedTags'])
redundancy_arr = ()

red_amazon_scraped_tags = 0 
for i in range(len(review_sentences)):
    temp_red = redundancy(review_sentences[i])
    red_amazon_scraped_tags += temp_red
    redundancy_arr = list(redundancy_arr)
    redundancy_arr.append((valid_asins[i],temp_red))
    redundancy_arr = tuple(redundancy_arr)
    
asins_count = len(valid_asins)

avd_red_amazon_scraped_tags = red_amazon_scraped_tags/asins_count

print('Avg reduncdancy NMF:', avd_red_amazon_scraped_tags)

['B0002SQ2P2', 'B0002KVQBA', 'B00020S7XK', 'B0002D03ZW', 'B0002WPSBC', 'B00006JN3G', 'B0002CZHN6', 'B00004T8R2', 'B00007M1TZ', 'B0002UPGOI', 'B000204SWE', 'B000629GES', 'B00017LSPI', 'B0002UM0JW', 'B0000C3GWU', 'B0001NNLHK', 'B0000BYDKO', 'B00066HP7Y', 'B0000AQIFW', 'B00066EK2W', 'B00005NIMJ', 'B00009WQS1', 'B0000DJEK7', 'B00028D778', 'B00030CHRQ', 'B0002IOIMQ', 'B0002Y5WXO', 'B00062UW5A', 'B00007GQLU', 'B00004TS16', 'B00005QXWI', 'B00018MSNI']
Avg reduncdancy NMF: 0.06690723690841899


In [18]:
print(asins_count)

32


In [19]:
print(redundancy_arr)

(('B0002SQ2P2', 0.01722706232293625), ('B0002KVQBA', 0.07407407407407396), ('B00020S7XK', 0.05678533309253431), ('B0002D03ZW', 0.06138624817088656), ('B0002WPSBC', 0.12073246487342737), ('B00006JN3G', 0.11009096747637015), ('B0002CZHN6', 0.044574556644966234), ('B00004T8R2', 0.0837725743386264), ('B00007M1TZ', 0.032730162085132464), ('B0002UPGOI', 0.05000000000000038), ('B000204SWE', 0.014196333273133965), ('B000629GES', 0.15912321167269505), ('B00017LSPI', 0.132598753895238), ('B0002UM0JW', 0.12080961137566348), ('B0000C3GWU', 0.06546032417026526), ('B0001NNLHK', 0.03273016208513235), ('B0000BYDKO', 0.042588999819400786), ('B00066HP7Y', 0.09693609377704371), ('B0000AQIFW', 0.03273016208513235), ('B00066EK2W', 0.04075900941810162), ('B00005NIMJ', 0.029716371096644156), ('B00009WQS1', 0.11381876958331827), ('B0000DJEK7', 6.661338147750939e-16), ('B00028D778', 0.01722706232293625), ('B00030CHRQ', 0.11886548438657696), ('B0002IOIMQ', 0.09263755863796663), ('B0002Y5WXO', 0.0681940619047632

In [34]:
#add emojis to topics
import emoji
for asin in valid_asins:
    topics, reviews = print_topics(asin)

    emoji_topics = []
    for topic_id, topic in enumerate(topics):

        # grab the average sentiment
        product_reviews_df = pd.DataFrame(reviews)

        product_reviews_df = product_reviews_df[product_reviews_df['topic'] == topic_id]
        polarity   = product_reviews_df['summary_sentiment'].mean()

        # append emojis to topic name based on range of sentiment
        if polarity <= -0.5:
            emoji_topics.append("😕 - "+topic)
        elif polarity > -0.5 and polarity < 0.5:
            emoji_topics.append("😐 - "+topic)
        else:
            emoji_topics.append("🙂 - "+topic)

    for emoji_topic in emoji_topics:
        print(emoji.emojize(emoji_topic))



d:\pycharmprojects\venv\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['make'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


return topics
Summary :
 24
Topics: 8
🙂 - bass sound music
🙂 - 2300 z
🙂 - great sound
🙂 - buy speakers computer
🙂 - remote satellite cable
🙂 - sound good card
😐 - work great bose
🙂 - speaker best set


d:\pycharmprojects\venv\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['make'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


return topics
Summary :
 24
Topics: 8
🙂 - bass sound music
😐 - 10 psw
😐 - port want sub
🙂 - great sound
🙂 - rca input subwoofer
🙂 - room live
😐 - home theater
😐 - price buy product


d:\pycharmprojects\venv\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['make'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


...

return topics
Summary :
 23
Topics: 8
🙂 - use picture camera
🙂 - larger thing card memory
🙂 - quality machine paper
😐 - work snorkel camera
🙂 - great overall think little camera
😐 - exposure camera stop
🙂 - glad shoot s100
😐 - charger reasonable battery especially


d:\pycharmprojects\venv\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['make'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


return topics
Summary :
 23
Topics: 8
🙂 - cd player turn
😐 - work plate screw
🙂 - mp3 far player rangeit price
😐 - purchase year little
😐 - pay make good
😐 - 1 3 2
😐 - accessory know player
🙂 - 100 90 250


d:\pycharmprojects\venv\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['make'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


return topics
Summary :
 23
Topics: 8
🙂 - cable sound headphone
🙂 - 650 hd
🙂 - comfortable headphones sound
🙂 - hd650 headphones ears
😐 - amp tube
😐 - years ago
🙂 - use like love
😐 - listen hear music


In [35]:
string = 'focal length wide open full frame image quality depth of field great lens highly recommend autofocus purple fringing chromatic aberration build quality shallow depth background blur'
lemmas = tokenizeText(string) 
lemmas = ' '.join(lemma for lemma in lemmas)
print(lemmas)
print(redundancy(lemmas))

topics, review = print_topics('B00007GQLU')
print(topics)
lemmas = ' '.join(lemma for lemma in topics)
print(lemmas)
print(redundancy(lemmas))

focal length wide open full frame image quality depth field great lens highly recommend autofocus purple fringe chromatic aberration build quality shallow depth background blur
0.03445412464587205


d:\pycharmprojects\venv\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['make'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


return topics
Summary :
 24
Topics: 8
['use lens shoot', '85mm f l', 'f1 8', 'lens love', 'purchase lens l', 'lenses image nice', 'light low', 'portraits great lens']
use lens shoot 85mm f l f1 8 lens love purchase lens l lenses image nice light low portraits great lens
0.10841440414140635
